# The ipyrad.analysis tool kit
Deren Eaton  


### Install software
All required software for this walkthrough is available on conda. 

In [1]:
# conda install -c ipyrad ipyrad structure clumpp bpp 
# conda install -c eaton-lab toytree toyplot
# conda install -c bioconda raxml

### Start an ipyparallel cluster
In a separate terminal run the following command to start a cluster of engines. If working on a notebook running remotely, use the dashboard to open a new terminal. 

In [49]:
# ipcluster start --n=4

You should then be able to connect to the engines in your notebook:

In [10]:
## connect to the cluster
import ipyparallel as ipp
ipyclient = ipp.Client()

## print number of engines
print len(ipyclient), "connected engines"

4 connected engines


### Assemble a RAD data set
The code here is to assemble the example empirical data set from the [ipyrad tutotial](http://ipyrad.readthedocs.io/userguide.html#empirical-examples). 

In [ ]:
## import ipyrad
import ipyrad as ip

Minimal workflow: scroll down for details.

In [ ]:
## create an Assembly object
data = ip.Assembly("simdata")

## set I/O paths for the data
data.set_params("project_dir", "~/workshop")
data.set_params("raw_fastq_path", "ipsimdata/rad_example_R1_.fastq.gz")
data.set_params("barcodes_path", "ipsimdata/rad_example_barcodes.txt")

## run all steps of the Assembly
data.run("1234567")

### Modify more parameters

In [3]:
## set params
data.set_params("filter_adapters", 2)
data.set_params("output_formats", "lpask")

## show params
data.get_params()

  0   assembly_name               simdata                                      
  1   project_dir                 /home/deren/workshop                         
  2   raw_fastq_path              ./ipsimdata/rad_example_R1_.fastq.gz         
  3   barcodes_path               ./ipsimdata/rad_example_barcodes.txt         
  4   sorted_fastq_path                                                        
  5   assembly_method             denovo                                       
  6   reference_sequence                                                       
  7   datatype                    rad                                          
  8   restriction_overhang        ('TGCAG', '')                                
  9   max_low_qual_bases          5                                            
  10  phred_Qscore_offset         33                                           
  11  mindepth_statistical        6                                            
  12  mindepth_majrule            6     

### Assemble the data set
You can run one or more steps just like in the CLI. 

In [4]:
## run all steps of assembly
data.run("1234567")


  Assembly: simdata
  [####################] 100%  sorting reads         | 0:00:03 | s1 | 
  [####################] 100%  writing/compressing   | 0:00:01 | s1 | 
  [####################] 100%  processing reads      | 0:00:03 | s2 | 
  [####################] 100%  dereplicating         | 0:00:00 | s3 | 
  [####################] 100%  clustering            | 0:00:01 | s3 | 
  [####################] 100%  building clusters     | 0:00:00 | s3 | 
  [####################] 100%  chunking              | 0:00:00 | s3 | 
  [####################] 100%  aligning              | 0:00:08 | s3 | 
  [####################] 100%  concatenating         | 0:00:00 | s3 | 
  [####################] 100%  inferring [H, E]      | 0:00:03 | s4 | 
  [####################] 100%  calculating depths    | 0:00:00 | s5 | 
  [####################] 100%  chunking clusters     | 0:00:00 | s5 | 
  [####################] 100%  consens calling       | 0:00:23 | s5 | 
  [####################] 100%  concat/shuffle input  | 0

### Access assembly results
You can easily access summary stats for the assembly as a data frame. 

In [5]:
## summary stats
data.stats

,state,reads_raw,reads_passed_filter,clusters_total,clusters_hidepth,hetero_est,error_est,reads_consens
1A_0,6,19862,19862,1000,1000,0.001852,0.000758,1000
1B_0,6,20043,20043,1000,1000,0.001900,0.000752,1000
1C_0,6,20136,20136,1000,1000,0.002084,0.000745,1000
1D_0,6,19966,19966,1000,1000,0.001803,0.000754,1000
2E_0,6,20017,20017,1000,1000,0.001831,0.000765,1000
2F_0,6,19933,19933,1000,1000,0.001996,0.000755,1000
2G_0,6,20030,20030,1000,1000,0.001940,0.000763,1000
2H_0,6,20199,20198,1000,1000,0.001747,0.000756,1000
3I_0,6,19885,19885,1000,1000,0.001818,0.000755,1000
3J_0,6,19822,19822,1000,1000,0.001968,0.000783,1000


### Plot statistics

In [35]:
import toyplot

## plot barplot
c, a, m = toyplot.bars(
    data.stats.hetero_est, 
    height=250, width=500,
    )

## style the axes
a.x.ticks.locator = toyplot.locator.Explicit(
    locations=range(len(data.stats)),
    labels=data.stats.index)
a.y.label.text = "Heterozygosity"
a.y.ticks.show = True

1A_0 1B_0 1C_0 1D_0 2E_0 2F_0 2G_0 2H_0 3I_0 3J_0 3K_0 3L_0 0.000 0.001 0.002 Heterozygosity 
 
 
 Save as .csv

### Access result files  
You can also access the stats files for each step, and the output files for downstream analyses. 

In [27]:
## s2 stats file
print data.stats_files.s2

## the .loci file location
print data.outfiles.loci

/home/deren/workshop/simdata_edits/s2_rawedit_stats.txt
/home/deren/workshop/simdata_outfiles/simdata.loci


## ipyrad.analysis tools
The ipyrad.analysis module includes many *wrapper* tools that can be used to efficiently run evolutionary analysis tools in a notebook. 

In [1]:
## import the toolkit
import ipyrad.analysis as ipa

### RAxML analysis
Simply enter the location of the phylip file, which can be accessed from the `.outfiles` attribute of the Assembly object. You can also provide a name and output directory, and set many other optional parameters. 

In [2]:
import ipyrad as ip
import ipyparallel as ipp

data = ip.load_json("/home/deren/workshop/simdata.json")
ipyclient = ipp.Client()


  loading Assembly: simdata
  from saved path: ~/workshop/simdata.json


Minimal workflow: scroll down for details.

In [3]:
## create a raxml object
s = ipa.raxml(
        name=data.name,
        phyfile=data.outfiles.phy, 
        workdir="~/workshop/analysis-raxml");

## run the analysis
s.run()

Error: set a new name for this job:
File exists: /home/deren/workshop/analysis-raxml/RAxML_info.simdata


### Modify parameters and other functions

In [4]:
## modify params
s.params.T = 4
s.params.N = 100

In [5]:
## print the raxml command as a string
print s.command

raxmlHPC-PTHREADS-SSE3 -f a -T 4 -m GTRGAMMA -N 100 -x 12345 -p 54321 -n simdata -w /home/deren/workshop/analysis-raxml -s /home/deren/workshop/simdata_outfiles/simdata.phy


In [6]:
## overwrite existing result with this 'name'
s.run(force=True)

job simdata finished successfully


### Access the tree files and plot

In [7]:
print s.trees

bestTree                   ~/workshop/analysis-raxml/RAxML_bestTree.simdata
bipartitions               ~/workshop/analysis-raxml/RAxML_bipartitions.simdata
bipartitionsBranchLabels   ~/workshop/analysis-raxml/RAxML_bipartitionsBranchLabels.simdata
boostrap                   ~/workshop/analysis-raxml/RAxML_bootstrap.simdata
info                       ~/workshop/analysis-raxml/RAxML_info.simdata



In [8]:
import toytree
tre = toytree.tree(s.trees.bipartitions)
tre.root(wildcard='3')
tre.draw(
    width=300,
    node_labels=tre.get_node_values("support"),
    node_size=20,
    );

3L_0 3K_0 3J_0 3I_0 2H_0 2G_0 2E_0 2F_0 1D_0 1C_0 1B_0 1A_0 100 100 100 100 100 100 100 100 100 100 
 
 
 Save as .csv

### introgression (abba-baba) analysis
The baba object can be used to set up abba-baba tests, to calculate results, and to generate plots to visualize them. 

Minimal example, scroll down for details. 

In [9]:
## create a baba object
b = ipa.baba(data=data.outfiles.loci)

## generate tests given the rooted tree
b.tests = [
    {"p4":["3L_0"], 
     "p3":["2F_0"], 
     "p2":["1D_0"],
     "p1":["1A_0"]}]

## run jobs distributed across the cluster
b.run(ipyclient)
b.results_table

[####################] 100%  calculating D-stats  | 0:00:00 |  


,dstat,bootmean,bootstd,Z,ABBA,BABA,nloci
0,-0.098,-0.098,0.228,0.43,11.5,14.0,1000


### Auto-generate tests 
Instead of writing out many tests explicitly, you can instead enter a rooted tree to the baba object and use this function to auto-generate four-taxon test fitting the tree and constraints. 

In [10]:
## init baba object
b = ipa.baba(data=data.outfiles.loci, newick=tre)

## generate all possible tests on this tree
b.generate_tests_from_tree()

## set constraints on tests
cdict = {"p4": ["3L_0"], 
         "p3": ["2E_0", "2F_0"], 
         "p2": ["1D_0"]}

## generate constrainted number of tests
b.generate_tests_from_tree(
    constraint_dict=cdict,
    constraint_exact=False,
    )

674 tests generated from tree
12 tests generated from tree


### Run all tests linked to a baba object

In [11]:
## run the tests (in this case 4) linked to the baba object
b.run(ipyclient)

## show results table
b.results_table

[####################] 100%  calculating D-stats  | 0:00:03 |  


,dstat,bootmean,bootstd,Z,ABBA,BABA,nloci
0,-0.181,-0.186,0.217,0.836,11.0,15.875,1000
1,-0.030,-0.037,0.213,0.143,13.0,13.812,1000
2,-0.005,-0.004,0.216,0.021,13.5,13.625,1000
3,-0.057,-0.060,0.222,0.255,12.5,14.000,1000
4,-0.135,-0.128,0.224,0.603,12.0,15.750,1000
5,-0.004,-0.007,0.208,0.021,14.0,14.125,1000
6,0.009,0.007,0.211,0.041,14.5,14.250,1000
7,-0.018,-0.009,0.227,0.080,13.5,14.000,1000
8,-0.231,-0.226,0.224,1.030,10.0,16.000,1000
9,-0.059,-0.076,0.211,0.279,12.0,13.500,1000


### Plot results 

In [12]:
b.plot(
    height=350,
    pct_tree_x = 0.4,
    pct_tree_y = 0.2,
    );

1 2 3 4 5 6 7 8 9 10 11 12 3L_0 3K_0 3J_0 3I_0 2H_0 2G_0 2E_0 2F_0 1D_0 1C_0 1B_0 1A_0 3.0 0.0 -0.8 0.8 Z-scores D-statistics <polygon points="81.818181818181827,70.199472974629245 85.007974481658692,70.199472974629245 88.197767145135586,70.199472974629245 91.387559808612451,70.199472974629245 94.577352472089331,70.199472974629245 97.767145135566182,70.199472974629245 100.95693779904309,70.199472974629245 104.14673046251995,70.199472974629245 107.33652312599681,70.199472974629245 110.5263157894737,70.199472974629245 113.71610845295058,70.199472974629245 116.90590111642746,70.199472974629245 120.09569377990431,70.199472974629245 123.2854864433812,70.199472974629245 126.47527910685807,70.199472974629245 129.66507177033495,70.199472974629245 132.85486443381183,70.199472974629245 136.04465709728871,70.199472974629245 139.23444976076556,70.199472974629245 142.42424242424244,70.199472974629245 145.61403508771932,70.199472974629245 148.8038277511962,70.199472974629245 151.99362041467305,70.199472974629245 155.18341307814995,70.199472974629245 158.3732057416268,70.199472974629245 161.56299840510368,70.199472974629245 164.75279106858056,70.199472974629245 167.94258373205741,70.199472974629245 171.13237639553429,70.199472974629245 174.32216905901117,70.199472974629245 174.32216905901117,270.86285083956369 171.13237639553429,270.86285083956369 167.94258373205741,270.86285083956369 164.75279106858056,270.86285083956369 161.56299840510368,270.86285083956369 158.3732057416268,270.86285083956369 155.18341307814995,270.86285083956369 151.99362041467305,270.86285083956369 148.8038277511962,270.86285083956369 145.61403508771932,270.86285083956369 142.42424242424244,270.86285083956369 139.23444976076556,270.86285083956369 136.04465709728871,270.86285083956369 132.85486443381183,270.86285083956369 129.66507177033495,270.86285083956369 126.47527910685807,270.86285083956369 123.2854864433812,270.86285083956369 120.09569377990431,270.86285083956369 116.90590111642746,270.86285083956369 113.71610845295058,270.86285083956369 110.5263157894737,270.86285083956369 107.33652312599681,270.86285083956369 104.14673046251995,270.86285083956369 100.95693779904309,270.86285083956369 97.767145135566182,270.86285083956369 94.577352472089331,270.86285083956369 91.387559808612451,270.86285083956369 88.197767145135586,270.86285083956369 85.007974481658692,270.86285083956369 81.818181818181827,270.86285083956369" style="fill:rgb(100%,100%,100%);fill-opacity:1.0;opacity:1.0;stroke:none"> <polygon points="81.818181818181827,270.86285083956369 85.007974481658692,270.86285083956369 88.197767145135586,270.86285083956369 91.387559808612451,270.86285083956369 94.577352472089331,270.86285083956369 97.767145135566182,270.86285083956369 100.95693779904309,270.86285083956369 104.14673046251995,270.86285083956369 107.33652312599681,270.86285083956369 110.5263157894737,270.86285083956369 113.71610845295058,270.86285083956369 116.90590111642746,270.86285083956369 120.09569377990431,270.86285083956369 123.2854864433812,270.86285083956369 126.47527910685807,270.86285083956369 129.66507177033495,270.86285083956369 132.85486443381183,270.86285083956369 136.04465709728871,270.86285083956369 139.23444976076556,270.86285083956369 142.42424242424244,270.86285083956369 145.61403508771932,270.86285083956369 148.8038277511962,270.86285083956369 151.99362041467305,270.86285083956369 155.18341307814995,270.86285083956369 158.3732057416268,270.86285083956369 161.56299840510368,270.86285083956369 164.75279106858056,270.86285083956369 167.94258373205741,270.86285083956369 171.13237639553429,270.86285083956369 174.32216905901117,270.86285083956369 174.32216905901117,270.86285083956369 171.13237639553429,270.86285083956369 167.94258373205741,270.86285083956369 164.75279106858056,270.86285083956369 161.56299840510368,270.44439348591897 158.3732057416268,270.30490770137072 155.18341307814995,270.16542191682254 151.99362041467305,269.60747877862957 148.8038277511962,269.04953564043666 145.614035087719

In [15]:
### Save the plot 
import toyplot.pdf
canvas, axes, mark = b.plot(height=350, pct_tree_x=0.4, pct_tree_y=0.2)
toyplot.pdf.render(canvas, "/home/deren/workshop/abba-baba.pdf")

## save the results table
b.results_table.to_csv("~/workshop/abba-baba.csv", sep="\t")

### Species tree inference by phylogenetic invariants
The program tetrad follows the algorithm of SVDquartets by inferring all possible quartet trees from a large SNP alignment and uses the program quartet maxcut (Snir et al. 2012) to infer a species tree by quartet joining. 

In [16]:
## create a tetrad class object
tet = ipa.tetrad(
    name=data.name,
    seqfile=data.outfiles.snpsphy,
    mapfile=data.outfiles.snpsmap,
    workdir="~/workshop/analysis-tetrad",
    nboots=100
    )

## run the analysis
tet.run(ipyclient)

loading seq array [12 taxa x 4314 bp]
max unlinked SNPs per quartet (nloci): 984
host compute node: [4 cores] on oud
inferring 495 induced quartet trees
[####################] 100%  initial tree | 0:00:00 |  
[####################] 100%  boot 100     | 0:00:40 |  


### Access result tetrad trees and draw

In [17]:
tet.trees

boots                /home/deren/workshop/analysis-tetrad/simdata.boots
cons                 /home/deren/workshop/analysis-tetrad/simdata.cons
nhx                  /home/deren/workshop/analysis-tetrad/simdata.nhx
tree                 /home/deren/workshop/analysis-tetrad/simdata.tree

In [63]:
## load unrooted result tree with toytree and draw
tre = toytree.tree(tet.trees.cons)
tre.draw(
    node_labels=tre.get_node_values("support"),
    node_size=20,
    );

1D_0 1C_0 1A_0 1B_0 2H_0 2G_0 2E_0 2F_0 3L_0 3K_0 3I_0 3J_0 100 100 100 100 100 100 100 100 100 
 
 
 Save as .csv

### Infer a species tree with BPP

In [ ]:
# conda install bpp -c ipyrad

In [19]:
## setup: define how samples group into 'species'
IMAP = {
    "1": ["1A_0", "1B_0", "1C_0"],
    "D": ["1D_0"],
    "2": ["2F_0", "2E_0", "2G_0"],
    "H": ["2H_0"],
    "3": ["3J_0", "3I_0", "3K_0"],
    "L": ["3L_0"],    
}

## setup: define a guidetree
GUIDE = "(((1,D),(2,H)),(3,L));"

## init a bpp object 
bpp = ipa.bpp(
    locifile=data.outfiles.loci, 
    imap=IMAP, 
    guidetree=GUIDE,
    workdir="~/workshop/analysis-bpp"
    );

## submit jobs to run on the cluster
bpp.submit_bpp_jobs("A00", nreps=2, ipyclient=ipyclient)

submitted 2 bpp jobs [A00] (1000 loci)


### Set parameters and filters
You can define all of the parameter settings that will be used in the BPP .ctl file by modifying the .params attributes. Similarly, you can modify which loci will be included in the analysis using the .filters attributes. 

In [20]:
## set some parameters 
bpp.params.burnin = 1000
bpp.params.nsample = 5000
bpp.params.infer_sptree = 1
bpp.params.infer_delimit = 0 

## set some filters
bpp.filters.maxloci = 200
bpp.filters.minsnps = 2

## submit jobs to run on the cluster
bpp.submit_bpp_jobs("A00", nreps=2, ipyclient=ipyclient)

submitted 2 bpp jobs [A00] (200 loci)


### Track running jobs
Unlike some of the other ipyrad.analysis tools, the bpp object does not "block" while the jobs are running. Meaning that after it sends jobs to run on the cluster you can continue to interact with the notebook. This is useful since BPP is not multi-threaded, so you will likely want to submit many different types of jobs. You can check on running jobs like below. 

In [21]:
## a list of submitted jobs
print bpp.asyncs

## a list of result files produced by jobs
print bpp.files

[<AsyncResult: _call_bpp>, <AsyncResult: _call_bpp>, <AsyncResult: _call_bpp>, <AsyncResult: _call_bpp>]
locifile    ~/workshop/simdata_outfiles/simdata.loci
mcmcfiles   ['~/workshop/analysis-bpp/A00-r0.mcmc.txt', '~/workshop/analysis-bpp/A00-r1.mcmc.txt']
outfiles    ['~/workshop/analysis-bpp/A00-r0.out.txt', '~/workshop/analysis-bpp/A00-r1.out.txt']



### Structure analyses

In [1]:
import ipyrad as ip
import ipyrad.analysis as ipa
import ipyparallel as ipp

data = ip.load_json("/home/deren/workshop/simdata.json")
ipyclient = ipp.Client()

  loading Assembly: simdata
  from saved path: ~/workshop/simdata.json


In [2]:
# conda install structure -c ipyrad
# conda install clumpp -c ipyrad

In [3]:
## create a structure class object
s = ipa.structure(
    name=data.name,
    strfile=data.outfiles.str,
    mapfile=data.outfiles.snpsmap,
    workdir="~/workshop/analysis-structure",    
    );

s.mainparams.burnin = 100
s.mainparams.numreps = 1000

## submit jobs to run on the cluster
for kpop in [2, 3, 4, 5]:
    s.submit_structure_jobs(kpop=kpop, nreps=5, ipyclient=ipyclient)

['simdata', '/home/deren/workshop/analysis-structure', 572500868, 12, 984, 2, 0]
['simdata', '/home/deren/workshop/analysis-structure', 709458039, 12, 984, 2, 1]
['simdata', '/home/deren/workshop/analysis-structure', 292133950, 12, 984, 2, 2]
['simdata', '/home/deren/workshop/analysis-structure', 316688770, 12, 984, 2, 3]
['simdata', '/home/deren/workshop/analysis-structure', 265959297, 12, 984, 2, 4]


submitted 5 structure jobs [simdata-K-2]


['simdata', '/home/deren/workshop/analysis-structure', 572500868, 12, 984, 3, 0]
['simdata', '/home/deren/workshop/analysis-structure', 709458039, 12, 984, 3, 1]
['simdata', '/home/deren/workshop/analysis-structure', 292133950, 12, 984, 3, 2]
['simdata', '/home/deren/workshop/analysis-structure', 316688770, 12, 984, 3, 3]
['simdata', '/home/deren/workshop/analysis-structure', 265959297, 12, 984, 3, 4]


submitted 5 structure jobs [simdata-K-3]


['simdata', '/home/deren/workshop/analysis-structure', 572500868, 12, 984, 4, 0]
['simdata', '/home/deren/workshop/analysis-structure', 709458039, 12, 984, 4, 1]
['simdata', '/home/deren/workshop/analysis-structure', 292133950, 12, 984, 4, 2]
['simdata', '/home/deren/workshop/analysis-structure', 316688770, 12, 984, 4, 3]
['simdata', '/home/deren/workshop/analysis-structure', 265959297, 12, 984, 4, 4]


submitted 5 structure jobs [simdata-K-4]


['simdata', '/home/deren/workshop/analysis-structure', 572500868, 12, 984, 5, 0]
['simdata', '/home/deren/workshop/analysis-structure', 709458039, 12, 984, 5, 1]
['simdata', '/home/deren/workshop/analysis-structure', 292133950, 12, 984, 5, 2]
['simdata', '/home/deren/workshop/analysis-structure', 316688770, 12, 984, 5, 3]
['simdata', '/home/deren/workshop/analysis-structure', 265959297, 12, 984, 5, 4]


submitted 5 structure jobs [simdata-K-5]


### Modify parameters settings

In [5]:
s.mainparams.burnin = 10000
s.mainparams.numreps = 100000
s.extraparams.usepopinfo = 0

### Summarize results with CLUMPP

In [6]:
## get results for a single K value
s.get_clumpp_table(3)

## make a dict for all results
tables = {}
for kpop in [2, 3, 4, 5]:
    tables[kpop] = s.get_clumpp_table(kpop)

[]


OSError: [Errno 2] No such file or directory